# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [ ]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

--2024-04-01 14:53:10--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M  --.-KB/s    in 0.1s    

2024-04-01 14:53:11 (199 MB/s) - ‘train.csv’ saved [29470338/29470338]



In [50]:
import pandas as pd
import nltk
df = pd.read_csv('train.csv')
def decrement(x):
  return x - 1
df.columns = ["label", "title", "lead"]
df["text"] = df["title"] + " " + df["lead"]
df["label"] = df["label"].apply(decrement)
df = df.sample(n=10000) # # only use 10K datapoints
df

,label,title,lead,text
71647,2,BellSouth Posts Lower Earnings,"BellSouth Corp. (BLS.N: Quote, Profile, Resear...",BellSouth Posts Lower Earnings BellSouth Corp....
50545,3,Who #39;s eavesdropping on your cell?,LONDON: If you thought that your little chit c...,Who #39;s eavesdropping on your cell? LONDON: ...
94867,0,Barroso Commission approved,The European Parliament has approved Jose Manu...,Barroso Commission approved The European Parli...
104550,3,Sun to buy remote-management firm,Sun Microsystems has signed a deal to acquire ...,Sun to buy remote-management firm Sun Microsys...
83863,0,"Aide: Arafat Not In Coma, In ICU",PARIS (CBS) A senior aide to Yasser Arafat sai...,"Aide: Arafat Not In Coma, In ICU PARIS (CBS) A..."
...,...,...,...,...
62089,2,Delta warns of bigger losses,No. 3 airline says loss to be worse than forec...,Delta warns of bigger losses No. 3 airline say...
22201,3,Ebbers' Lawyers Seek Immunity For Witnesses,Lawyers for the former WorldCom Inc. chief exe...,Ebbers' Lawyers Seek Immunity For Witnesses ...
62441,1,Astros Lead Cards 5-2 After Eight Innings,Houston Astros #39; Jeff Kent homers in the fi...,Astros Lead Cards 5-2 After Eight Innings Hous...
21410,0,Russia Offers \$10 Million for Chechen Rebels ...,MOSCOW (Reuters) - The Russia government has ...,Russia Offers \$10 Million for Chechen Rebels ...


## Hugginface Transformers

In [51]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

cpu


In [52]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text
model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder
import numpy as np
unique_labels, counts = np.unique(df["label"], return_counts=True)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_labels))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
##TODO print the summary of the model
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [54]:
##TODO split the sample into a training and a test set
##TODO prepare the dataset for torch.
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
X = tokenizer(list(df['text']), return_tensors="pt", truncation=True, padding=True)
y = torch.tensor(list(df['label']))


In [55]:
optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label'].tolist(), test_size=.2)

X_train, X_test, y_train, y_test = np.array(X_train[:1600]), np.array(X_test[:400]), np.array(y_train[:1600]), np.array(y_test[:400])
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_test, y_train, y_test = X_train.reshape(-1, 16), X_test.reshape(-1, 16), y_train.reshape(-1, 16), y_test.reshape(-1, 16) # create batches of 16
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_test = X_train.tolist(), X_test.tolist()

(1600,) (400,) (1600,) (400,)
(100, 16) (25, 16) (100, 16) (25, 16)


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 71647 to 96732
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  int64 
 1   title   10000 non-null  object
 2   lead    10000 non-null  object
 3   text    10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 390.6+ KB


In [58]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)

In [70]:
from tqdm import tqdm
import torch
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

num_epochs = 1
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        labels = torch.tensor(labels).long().to(device)
        model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
        outputs = model(**model_inputs, labels = labels)
        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

100%|██████████| 100/100 [13:39<00:00,  8.19s/it]


In [71]:
predictions, targets = [], []
model.eval()

with torch.no_grad():
    for text, labels in tqdm(zip(X_test, y_test), total=len(X_test)):
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        model_inputs = {k:v.to(device) for k,v in model_inputs.items()}

        output = model(**model_inputs)
        logits = output[0]
        predictions.extend(logits.argmax(dim=1).tolist())
        targets.extend(labels)

from sklearn import metrics
accuracy = metrics.accuracy_score(targets, predictions)
print ("accuracy", accuracy)
classification_report = metrics.classification_report(targets, predictions)
print (classification_report)

100%|██████████| 25/25 [01:01<00:00,  2.46s/it]

accuracy 0.84
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       104
           1       0.96      0.88      0.92        77
           2       0.80      0.86      0.83       124
           3       0.82      0.75      0.78        95

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



# Doc Embedding

In [72]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip
!unzip sts2017.gs.zip

--2024-04-01 15:57:36--  http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Resolving alt.qcri.org (alt.qcri.org)... 37.186.61.205
Connecting to alt.qcri.org (alt.qcri.org)|37.186.61.205|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip [following]
--2024-04-01 15:57:36--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Connecting to alt.qcri.org (alt.qcri.org)|37.186.61.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87902 (86K) [application/zip]
Saving to: ‘sts2017.eval.v1.1.zip’

sts2017.eval.v1.1.z 100%[===================>]  85.84K  --.-KB/s    in 0.1s    

2024-04-01 15:57:37 (684 KB/s) - ‘sts2017.eval.v1.1.zip’ saved [87902/87902]

--2024-04-01 15:57:37--  http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip
Resolving alt.qcri.org (alt.qcri.org)... 37.186.61.205
Connecting to a

In [73]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]

    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

('A person is on a baseball team.',
 'A person is playing basketball on a team.',
 2.4)

In [74]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [75]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [80]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

def infer_vectors(model, tagged_texts):
    return [model.infer_vector(doc.words) for doc in tagged_texts]

def create_tagged_documents(texts):
    tagged_texts = [TaggedDocument(words=text.split(), tags=[i]) for i, text in enumerate(texts)]
    return tagged_texts

tagged_text_a = create_tagged_documents(text_a)
tagged_text_b = create_tagged_documents(text_b)

##TODO train Doc2Vec on the texts in the dataset
model = Doc2Vec(vector_size=300, min_count=2, epochs=20)
model.build_vocab(tagged_text_a + tagged_text_b)
model.train(tagged_text_a + tagged_text_b, total_examples=model.corpus_count, epochs=model.epochs)

##TODO derive the word vectors for each text in the dataset
text_a_vectors = infer_vectors(model, tagged_text_a)
text_b_vectors = infer_vectors(model, tagged_text_b)

##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
predictions = [cosine_similarity(a, b) for a, b in zip(text_a_vectors, text_b_vectors)]
evaluate(predictions, labels)
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

spearman's rank correlation 0.2022986793129762


In [83]:
##TODO do the same with embeddings provided by spaCy
import spacy
nlp = spacy.load("en_core_web_sm")

def get_spacy_vectors(texts):
    return [nlp(text).vector for text in texts]

text_a_vectors_spacy = get_spacy_vectors(text_a)
text_b_vectors_spacy = get_spacy_vectors(text_b)

predictions_spacy = [cosine_similarity(a, b) for a, b in zip(text_a_vectors_spacy, text_b_vectors_spacy)]
evaluate(predictions_spacy, labels)

spearman's rank correlation 0.48796451382976114


In [86]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 533.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
##TODO do the same with SBERT embeddings
from sentence_transformers import SentenceTransformer
model = "bert-base-nli-mean-tokens"
sbert_model = SentenceTransformer(model)

text_a_vectors_sbert = sbert_model.encode(text_a)
text_b_vectors_sbert = sbert_model.encode(text_b)

predictions_sbert = [cosine_similarity(a, b) for a, b in zip(text_a_vectors_sbert, text_b_vectors_sbert)]
evaluate(predictions_sbert, labels)